In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('data/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' AS ColumnName, COUNT(*) AS CountEmpty
    FROM boston
    WHERE CRIM IS NULL
    UNION
    SELECT 'ZN', COUNT(*)
    FROM boston
    WHERE ZN IS NULL
    UNION
    SELECT 'INDUS', COUNT(*)
    FROM boston
    WHERE INDUS IS NULL
    UNION
    SELECT 'CHAS', COUNT(*)
    FROM boston
    WHERE CHAS IS NULL
    UNION
    SELECT 'NOX', COUNT(*)
    FROM boston
    WHERE NOX IS NULL;
    ''',
    con,
)

,ColumnName,CountEmpty
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT 'CRIM' AS ColumnName, COUNT(DISTINCT CRIM) AS CountUnique
    FROM boston
    UNION
    SELECT 'ZN', COUNT(DISTINCT ZN)
    FROM boston
    UNION
    SELECT 'INDUS', COUNT(DISTINCT INDUS)
    FROM boston
    UNION
    SELECT 'CHAS', COUNT(DISTINCT CHAS)
    FROM boston
    UNION
    SELECT 'NOX', COUNT(DISTINCT NOX)
    FROM boston;
    ''',
    con,
)

,ColumnName,CountUnique
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks+=1
pd.read_sql(
    '''
    WITH t AS (
        SELECT 'CRIM' as ColumnName, AVG(CRIM) AS Median
            FROM (SELECT CRIM
                FROM boston
                ORDER BY CRIM
                LIMIT (SELECT iif(COUNT(*) % 2 = 0, 2, 1) FROM boston)
                    OFFSET (SELECT iif(COUNT(*) % 2 = 0, COUNT(*) / 2 - 1, COUNT(*) / 2) FROM boston)
            )
        UNION
        SELECT 'ZN', AVG(ZN)
            FROM (SELECT ZN
                FROM boston
                ORDER BY ZN
                LIMIT (SELECT iif(COUNT(*) % 2 = 0, 2, 1) FROM boston)
                    OFFSET (SELECT iif(COUNT(*) % 2 = 0, COUNT(*) / 2 - 1, COUNT(*) / 2) FROM boston)
            )
        UNION
        SELECT 'INDUS', AVG(INDUS)
            FROM (SELECT INDUS
                FROM boston
                ORDER BY INDUS
                LIMIT (SELECT iif(COUNT(*) % 2 = 0, 2, 1) FROM boston)
                    OFFSET (SELECT iif(COUNT(*) % 2 = 0, COUNT(*) / 2 - 1, COUNT(*) / 2) FROM boston)
            )
        UNION
        SELECT 'CHAS', AVG(CHAS)
            FROM (SELECT CHAS
                FROM boston
                ORDER BY CHAS
                LIMIT (SELECT iif(COUNT(*) % 2 = 0, 2, 1) FROM boston)
                    OFFSET (SELECT iif(COUNT(*) % 2 = 0, COUNT(*) / 2 - 1, COUNT(*) / 2) FROM boston)
            )
        UNION
        SELECT 'NOX', AVG(NOX)
            FROM (SELECT NOX
                FROM boston
                ORDER BY NOX
                LIMIT (SELECT iif(COUNT(*) % 2 = 0, 2, 1) FROM boston)
                    OFFSET (SELECT iif(COUNT(*) % 2 = 0, COUNT(*) / 2 - 1, COUNT(*) / 2) FROM boston)
            )
    )
    
    SELECT *
    FROM t
    WHERE Median = (SELECT MIN(Median) FROM t);
    ''',
    con,
)

,ColumnName,Median
0,CHAS,0.0
1,ZN,0.0


**Вывод:**

В столбцах CHAS и ZN больше половины нулевых значений.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        u.NumHouses,
        ROUND(u.AVG_RM_EXPENSIVE, 3) AS AVG_RM_EXPENSIVE,
        ROUND(u.AVG_RM_CHEAP, 3) AS AVG_RM_CHEAP,
        ROUND(u.AVG_RM_EXPENSIVE - u.AVG_RM_CHEAP, 3) AS AVG_RM_DIFF
    FROM (
        SELECT
            25 AS NumHouses,
            AVG_RM_EXPENSIVE.RM AS AVG_RM_EXPENSIVE,
            AVG_RM_CHEAP.RM AS AVG_RM_CHEAP
        FROM
            (SELECT AVG(RM) AS RM FROM boston
                WHERE MEDV IN (
                    SELECT MEDV FROM boston
                    ORDER BY MEDV DESC
                    LIMIT 1)) AS AVG_RM_EXPENSIVE,
            (SELECT AVG(t.RM) AS RM FROM (
                SELECT RM FROM boston AS b
                ORDER BY MEDV ASC
                LIMIT 25) AS t) AS AVG_RM_CHEAP
        UNION
        SELECT
            50 AS Num, AVG_RM_EXPENSIVE.RM, AVG_RM_CHEAP.RM
        FROM
            (SELECT AVG(RM) AS RM FROM boston
                WHERE MEDV IN (
                    SELECT MEDV FROM boston
                    ORDER BY MEDV DESC
                    LIMIT 1)) AS AVG_RM_EXPENSIVE,
            (SELECT AVG(t.RM) AS RM FROM (
                SELECT RM FROM boston AS b
                ORDER BY MEDV ASC
                LIMIT 50) AS t) AS AVG_RM_CHEAP
        UNION
        SELECT
            100 AS Num, AVG_RM_EXPENSIVE.RM, AVG_RM_CHEAP.RM
        FROM
            (SELECT AVG(RM) AS RM FROM boston
                WHERE MEDV IN (
                    SELECT MEDV FROM boston
                    ORDER BY MEDV DESC
                    LIMIT 1)) AS AVG_RM_EXPENSIVE,
            (SELECT AVG(t.RM) AS RM FROM (
                SELECT RM FROM boston AS b
                ORDER BY MEDV ASC
                LIMIT 100) AS t) AS AVG_RM_CHEAP
        UNION
        SELECT
            200 AS Num, AVG_RM_EXPENSIVE.RM, AVG_RM_CHEAP.RM
        FROM
            (SELECT AVG(RM) AS RM FROM boston
                WHERE MEDV IN (
                    SELECT MEDV FROM boston
                    ORDER BY MEDV DESC
                    LIMIT 1)) AS AVG_RM_EXPENSIVE,
            (SELECT AVG(t.RM) AS RM FROM (
                SELECT RM FROM boston AS b
                ORDER BY MEDV ASC
                LIMIT 200) AS t) AS AVG_RM_CHEAP
        UNION
        SELECT
            300 AS Num, AVG_RM_EXPENSIVE.RM, AVG_RM_CHEAP.RM
        FROM
            (SELECT AVG(RM) AS RM FROM boston
                WHERE MEDV IN (
                    SELECT MEDV FROM boston
                    ORDER BY MEDV DESC
                    LIMIT 1)) AS AVG_RM_EXPENSIVE,
            (SELECT AVG(t.RM) AS RM FROM (
                SELECT RM FROM boston AS b
                ORDER BY MEDV ASC
                LIMIT 300) AS t) AS AVG_RM_CHEAP
    ) AS u;
    ''',
    con,
)

,NumHouses,AVG_RM_EXPENSIVE,AVG_RM_CHEAP,AVG_RM_DIFF
0,25,7.484,5.748,1.736
1,50,7.484,5.753,1.731
2,100,7.484,5.887,1.597
3,200,7.484,5.912,1.572
4,300,7.484,5.972,1.512


**Вывод:**

Чем больше комнат в доме, тем меньше разница в стоимости между дорогими и дешевыми домами.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT t.LSTAT, t.RNK_LSTAT
    FROM (
        SELECT
            MEDV,
            LSTAT,
            RANK() OVER (ORDER BY LSTAT) AS RNK_LSTAT
        FROM boston
    ) AS t
    WHERE t.MEDV IN (SELECT MAX(MEDV) FROM boston);
    ''',
    con,
)

,LSTAT,RNK_LSTAT
0,1.73,1
1,1.92,2
2,2.88,6
3,2.96,8
4,2.97,9
5,3.16,15
6,3.26,17
7,3.32,18
8,3.70,26
9,3.73,27


**Вывод:**

В более дорогих домах процент населения с более низким статусом ниже.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        CASE
            WHEN CHAS > 0 THEN 'граничит'
            ELSE 'не граничит'
        END AS RIVER,
        ROUND(AVG(MEDV), 2) AS AVG_MEDV
    FROM boston AS b
    GROUP BY CHAS
    ORDER BY CHAS DESC;
    ''',
    con,
)

,RIVER,AVG_MEDV
0,граничит,28.44
1,не граничит,22.09


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT t.ColumnName
    FROM (
        SELECT 'CRIM' AS ColumnName, COUNT(*) AS CNT
        FROM boston AS b
        WHERE (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) > (SELECT AVG(CRIM) FROM boston)
        UNION
        SELECT 'ZN', COUNT(*) AS CNT
        FROM boston AS b
        WHERE (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) > (SELECT AVG(ZN) FROM boston)
        UNION
        SELECT 'INDUS', COUNT(*) AS CNT
        FROM boston AS b
        WHERE (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) > (SELECT AVG(INDUS) FROM boston)
        UNION
        SELECT 'NOX', COUNT(*) AS CNT
        FROM boston AS b
        WHERE (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) > (SELECT AVG(NOX) FROM boston)
    ) AS t
    WHERE t.CNT > 0;
    ''',
    con,
)

,ColumnName
0,INDUS
1,NOX


**Вывод:**

Около воды выше доля акров неторгового бизнеса, а также выше концентрация оксидов азота.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        ROUND(AVG(t.INDUS), 2) AS INDUS,
        ROUND(AVG(t.NOX), 2) AS NOX,
        t.PCT AS PERCENTILE
    FROM (
        SELECT
            INDUS, NOX, (NTILE(10) OVER (ORDER BY INDUS, NOX)) * 10 AS PCT
        FROM boston
    ) AS t
    GROUP BY t.PCT
    ORDER BY t.PCT;
    ''',
    con,
)

,INDUS,NOX,PERCENTILE
0,2.02,0.44,10
1,3.65,0.49,20
2,5.25,0.45,30
3,6.47,0.47,40
4,8.26,0.53,50
5,10.75,0.52,60
6,17.25,0.56,70
7,18.10,0.68,80
8,18.31,0.71,90
9,21.94,0.69,100


**Вывод:**

С увеличением доли промышленной застройки растет концентрации оксидов азота.

In [15]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
